In [2]:
#Langchain Implements document abstraction it has two attributes :- meta_Data,page_content,meta_Data captures the iformation about the source of the documents






In [3]:
from langchain_core.documents import Document



documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [4]:
documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.')]

In [5]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
groq_api_key = os.getenv("GROQ_API_KEY")
llm=ChatGroq(groq_api_key=groq_api_key,model="llama3-8b-8192")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001631233CE50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000016311DECDD0>, model_name='llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [8]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


c:\Users\aakash\Desktop\LCEL-LLM\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
#vectorstores(here documents converted to vector and then chroma used to store the vectors)
from langchain_chroma import Chroma
vectorstore = Chroma.from_documents(documents, embedding=embeddings)
vectorstore


In [12]:
vectorstore.similarity_search("cat")

[Document(id='4f2906c8-dee6-446f-9b37-0e6c359712f6', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='e2f965fc-b81d-4287-be08-6643fe8b7cfc', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='0bd0a887-076f-48c2-a342-24c2578c6b4b', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(id='a6b85ced-fc07-49e7-bcf4-7577698c6fe0', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [14]:
vectorstore.similarity_search_with_score("cat")

[(Document(id='4f2906c8-dee6-446f-9b37-0e6c359712f6', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.9351057410240173),
 (Document(id='e2f965fc-b81d-4287-be08-6643fe8b7cfc', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  1.574089765548706),
 (Document(id='0bd0a887-076f-48c2-a342-24c2578c6b4b', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
  1.5956902503967285),
 (Document(id='a6b85ced-fc07-49e7-bcf4-7577698c6fe0', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
  1.665792465209961)]

In [15]:
##retrievers -they re runnables so they implement a standaed set of methods (Retrieve the data from vectorstore)

In [16]:
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda
#this runnable lambda is applying to all the things present in this particualar batch
retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)#bind k=1 helps to get the top 1 result
retriever.batch(["cat","dog"])#first it will do vector similarity search by taking cat and then dog

[[Document(id='4f2906c8-dee6-446f-9b37-0e6c359712f6', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='e2f965fc-b81d-4287-be08-6643fe8b7cfc', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [17]:
#VectorStores implement As a retriever method that will generate a retriever specifically a vector store retrieval These retrievers include specific search type and search quog attribute that identify what methods of the underlying vector stores to call and how to parameterize them

In [18]:
vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)
retriever.batch(["cat", "dog"])

[[Document(id='4f2906c8-dee6-446f-9b37-0e6c359712f6', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='e2f965fc-b81d-4287-be08-6643fe8b7cfc', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [19]:
#integrate this retrriever along with the chain
## RAG(Retrieval Augmented Generation)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
#here in message we have give two inputs one is question and other is context
message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""
prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain={"context":retriever,"question":RunnablePassthrough()}|prompt|llm#we re going to get the information in the invoke method thats why we write runnabepassthrough

response=rag_chain.invoke("tell me about dogs")
print(response.content)

Based on the provided context, here's what can be said about dogs:

Dogs are great companions, known for their loyalty and friendliness.
